In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime
import math

# Script summary
- Import many xml files that were generated by the Send to Roadnet process in D365
- Perform processing on the XML files, and import them into a Pandas dataframe containing outbound order lines from D365
- Convert this into a dataframe that emulates the data that Roadnet would have returned - apply spec for every field in the file
- Import a list of SessionIDs, one for each user (or virtual user) that will be importing and posting an inbound file from Roadnet
- Split the inbound file (from Roadnet) into one file per SessionID
- Export as CSV (take note lines must end with CR-NL)

### Clean up temporary folders

In [2]:
# Set filenames
consolidated_roadnet_out_file = 'data/roadnet/xml_consolidated/consolidated_roadnet_out.xml'

In [3]:
try:
    os.remove(consolidated_roadnet_out_file)
except:
    pass
    #print(consolidated_roadnet_out_file + ' not in folder')

In [4]:
# Clean up the xml_prep folder
path = "data/roadnet/xml_prep/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)

# Clean up the xml_consolidated folder
path = "data/roadnet/xml_consolidated/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)    

### Convert D365 XML outbound files from a single line (with no newlines) to multiple lines by inserting a newline after evert '>' character
- Store the resulting files in the /xml_prep/ folder

In [5]:
# Get the list of all files and directories
path = "data/roadnet/xml_outbound/"
dir_list = os.listdir(path)

print("Loading " + str(len(dir_list)) + " XML files in ", path)


Loading 73 XML files in  data/roadnet/xml_outbound/


In [6]:
def xml_to_multiple_lines(fname, fnum):

    with open(fname, 'r') as fr:
        # reading line by line
        lines = fr.readlines()
        last_line = len(lines)
        #print(last_line)

    for line in lines:
        replaced_line = re.sub(">", ">\u000A", line)

    outfile = 'data/roadnet/xml_prep/f'+str(fnum)+'.xml'

    with open(outfile, 'w') as fw:
        fw.write(replaced_line)    

    return

In [7]:
# Convert xml from D365 by adding newlines

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    #print(xml_file)
    xml_to_multiple_lines(xml_file,i)

number_of_xml = len(dir_list)

### Consolidate the processed XML outbound files into a single, consolidated xml file
- Remove all lines that are not transaction line items

In [8]:
# Get the list of all files and directories
path = "data/roadnet/xml_prep/"
prep_dir_list = os.listdir(path)

In [9]:
def import_roadnet_files2(fname, fnum, outfile):

    try:    
        with open(fname, 'r') as fr:
            # reading line by line
            lines = fr.readlines()

            last_line = len(lines)

            # opening in writing mode
            with open(outfile, 'a') as fw:
                for line in lines:      
                    substr1 = 'CCBROADNETWORKBENCHSESSIONTABLEENTITY'       
                    x1 = re.search(substr1, line)
                    substr2 = 'Document>'       
                    x2 = re.search(substr2, line)
                    substr3 = 'xml version='       
                    x3 = re.search(substr3, line)
                    #print(x)
                    if x1 == None and x2 == None and x3 == None:
                        fw.write(line)
        #print(fname+" lines deleted")

    except:
        print("Error importing "+fname)

In [10]:
for i in range(0,len(prep_dir_list)):
    xml_file = path+prep_dir_list[i]
    #print(i)
    #print(xml_file)
    import_roadnet_files2(xml_file,i, consolidated_roadnet_out_file)

### Split up the consolidated xml file into 5,000 lines, else they cannot be imported into a Pandas dataframe
- Store these files in the same folder as the consolidated xml, and delete the consolidated xml after the split
- Add the lines to turn this into a valid XML format

In [11]:
fname = consolidated_roadnet_out_file
outfile = fname

sizelimit = 5000

try:    
    with open(fname, 'r') as fr:
        # reading line by line
        lines = fr.readlines()

        last_line = len(lines)

        line_counter = 1

        # opening in writing mode
        last_x = 0
        for i in range(0,len(lines)):
            line = lines[i]
            if i == last_line:
                print(line)
            x = int(i/sizelimit)
            with open(outfile[:-4]+str(x)+'.xml', 'a') as fw:        
                if i == 0:
                    fw.write('<?xml version="1.0" encoding="utf-8"?>\n')
                    fw.write('<Document>\n')
                if x > last_x:
                    last_x = x
                    fw.write('<?xml version="1.0" encoding="utf-8"?>\n')
                    fw.write('<Document>\n')
                fw.write(line)
                if int((i+1)/sizelimit) > x:
                    fw.write('</Document>')
                if i == len(lines) - 1:
                    fw.write('</Document>')
    os.remove(consolidated_roadnet_out_file)
                
except:
    print("Error importing "+fname)

### Import the transformed XML files into a Pandas dataframe

In [12]:
# Get the list of all files and directories
path = "data/roadnet/xml_consolidated/"
dir_list = os.listdir(path)

#print("Files and directories in '", path, "' :")

# prints all files
#print(dir_list)

In [13]:
#try1 = pd.read_xml("./data/roadnet/xml_consolidated/consolidated_roadnet_out0.xml")

In [14]:
# Import the first file into a dataframe
rdnet_out = pd.read_xml(path+dir_list[0])

In [15]:
# Import the rest of the files, and append to the dataframe
for i in range(1,len(dir_list)):
    xml_file = path+dir_list[i]
    print(str(i)+xml_file)
    temp = pd.read_xml(path+dir_list[i])
    rdnet_out = pd.concat([rdnet_out, temp], ignore_index=True)

1data/roadnet/xml_consolidated/consolidated_roadnet_out5.xml
2data/roadnet/xml_consolidated/consolidated_roadnet_out9.xml
3data/roadnet/xml_consolidated/consolidated_roadnet_out3.xml
4data/roadnet/xml_consolidated/consolidated_roadnet_out0.xml
5data/roadnet/xml_consolidated/consolidated_roadnet_out1.xml
6data/roadnet/xml_consolidated/consolidated_roadnet_out4.xml
7data/roadnet/xml_consolidated/consolidated_roadnet_out6.xml
8data/roadnet/xml_consolidated/consolidated_roadnet_out2.xml
9data/roadnet/xml_consolidated/consolidated_roadnet_out7.xml
10data/roadnet/xml_consolidated/consolidated_roadnet_out8.xml


In [16]:
len(rdnet_out)

52710

In [17]:
rdnet_out[rdnet_out.duplicated(['INVENTTRANSID'], keep=False)]

,INVENTTRANSID,BLOCKEDSTATUS,CITY,CREDITRELEASEDATE,CREDITRELEASETIME,DAILYURGENCYINCREASE,DEPENDENCY,DESCRIPTION,DYNAMICSINTERNALSESSIONID,ERROR,...,REFTABLEID,ROADNETROUTE,SALESMANAGER,SHIPDATE,URGENCY,VOLUME,WAREHOUSEDESC,WAREHOUSEID,WEIGHT,WEIGHTUNIT
0,ZA1-108660005,0,Polokwane,1900-01-01T00:00:00+00:00,0,0,0,Pretoria Finished Goods WH,ZA1-000021129,NaN,...,4067,ZA014B00,NaN,2023-08-17T00:00:00+00:00,0,1.641714e+10,Pretoria Finished Goods WH,ZA014B,9388.8,CS
1,ZA1-108660009,0,Polokwane,1900-01-01T00:00:00+00:00,0,0,0,Pretoria Finished Goods WH,ZA1-000021129,NaN,...,4067,ZA014B00,NaN,2023-08-17T00:00:00+00:00,0,3.098124e+10,Pretoria Finished Goods WH,ZA014B,17702.4,CS
2,ZA1-108660013,0,Polokwane,1900-01-01T00:00:00+00:00,0,0,0,Pretoria Finished Goods WH,ZA1-000021129,NaN,...,4067,ZA014B00,NaN,2023-08-17T00:00:00+00:00,0,0.000000e+00,Pretoria Finished Goods WH,ZA014B,9432.0,CS
3,ZA1-101133446,0,Nongoma,1900-01-01T00:00:00+00:00,0,0,0,BROWNS NONGOMA,ZA1-000021135,NaN,...,1328,ZA032B03,NaN,2023-08-17T00:00:00+00:00,0,3.190988e+08,Phoenix Finished Goods WH,ZA032B,160.5,CS
4,ZA1-101134190,0,Nongoma,1900-01-01T00:00:00+00:00,0,0,0,BROWNS NONGOMA,ZA1-000021135,NaN,...,1328,ZA032B03,NaN,2023-08-17T00:00:00+00:00,0,1.270080e+08,Phoenix Finished Goods WH,ZA032B,86.0,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52705,ZA1-109297498,0,NaN,1900-01-01T00:00:00+00:00,0,0,0,NaN,ZA1-000021190,NaN,...,1328,ZA014B02,NaN,1900-01-01T00:00:00+00:00,0,0.000000e+00,NaN,NaN,0.0,KG
52706,ZA1-109297499,0,NaN,1900-01-01T00:00:00+00:00,0,0,0,NaN,ZA1-000021190,NaN,...,1328,ZA014B02,NaN,1900-01-01T00:00:00+00:00,0,0.000000e+00,NaN,NaN,0.0,KG
52707,ZA1-109297500,0,NaN,1900-01-01T00:00:00+00:00,0,0,0,NaN,ZA1-000021190,NaN,...,1328,ZA014B02,NaN,1900-01-01T00:00:00+00:00,0,0.000000e+00,NaN,NaN,0.0,KG
52708,ZA1-109297501,0,NaN,1900-01-01T00:00:00+00:00,0,0,0,NaN,ZA1-000021190,NaN,...,1328,ZA014B02,NaN,1900-01-01T00:00:00+00:00,0,0.000000e+00,NaN,NaN,0.0,KG


In [18]:
rdnet_out.drop(columns={'OUTPERFORMROADNETDESTINATION'}, inplace=True, axis=1)

In [19]:
#rdnet_out.query("INVENTTRANSID == 'ZA1-108660005'").to_excel("./data/roadnet/duplicates.xlsx")

In [20]:
rdnet_out = rdnet_out.drop_duplicates(keep='first').copy()

In [21]:
len(rdnet_out)

4663

In [22]:
#rdnet_out = rdnet_out.drop_duplicates(subset = ['INVENTTRANSID'], keep='first').copy()

### Create the Roadnet inbound file by copying selected columns as-is from the outbound data

In [23]:
rdnet_in = rdnet_out[['QUANTITY','LOCATIONID','INVENTTRANSID','ITEMID','ORDERID','WAREHOUSEID','PRODUCTNAME','ROADNETROUTE','ORDERACCOUNT','ORDERACCOUNTNAME','WEIGHT']].copy()

In [24]:
rdnet_in = rdnet_in.dropna(subset=['WAREHOUSEID']).copy()

In [25]:
#rdnet_in[rdnet_in. duplicated(subset = ['INVENTTRANSID'], keep=False)]

In [26]:
#rdnet_in[rdnet_in. duplicated(subset = ['INVENTTRANSID'], keep=False)].to_excel("./data/roadnet/rdnet_in_dup.xlsx")

In [27]:
rdnet_in.rename(columns={'QUANTITY':'CASEQTY','LOCATIONID':'DESTINATIONLOCATIONID','ORDERID':'ORDERNUMBER','WAREHOUSEID':'ORIGINLOCATIONID','ORDERACCOUNT':'STOPLOCATIONID','ORDERACCOUNTNAME':'STOPLOCATIONNAME'}, inplace=True)

### Create the rest of the fields as per the Roadnet inbound file spec

In [28]:
today = str(datetime.now())
today = today.replace(':','h')
today = today.replace('-','')
today = today.replace(' ','-')
today = today[0:14] + '-'
#print("Today date is: ", today)

In [29]:
rdnet_in['ROADNETROUTEINTERNALROUTEID'] = today + rdnet_in['STOPLOCATIONID'].astype(str)

In [30]:
no_of_customers = len(rdnet_in['STOPLOCATIONID'].unique())

In [31]:
rdnet_in['APPTID'] = ''
rdnet_in['DESCRIPTION'] = 'BLOEM_PLAN'
rdnet_in['ERROR'] = ''
rdnet_in['FIRSTDRIVER'] = '825196'
rdnet_in['FIRSTTRAILER'] = 'ST29PTAIL'
rdnet_in['LASTSTOPISDESTINATION'] = 'No'
rdnet_in['LOADID'] = ''
rdnet_in['LOADTEMPLATEID'] = ''
rdnet_in['ORDERTYPE'] = 'rotOrder'
rdnet_in['ORIGINDESTINATION'] = 'Yes'
rdnet_in['PALLETQTY'] = '0'
rdnet_in['REFERENCECATEGORY'] = 'Sales'
rdnet_in['REFERENCEDOCUMENT'] = 'SalesOrder'
rdnet_in['ROADNETINTERNALSESSIONID'] = '35411'
rdnet_in['ROADNETREGIONID'] = 'ZA1'
rdnet_in['ROUTECODE'] = ''
rdnet_in['SECONDDRIVER'] = ''
rdnet_in['SECONDTRAILER'] = ''
rdnet_in['SEQUENCEDISTANCE'] = '.000000'
rdnet_in['SEQUENCENUMBER'] = '1'
rdnet_in['SEQUENCETRAVELTIME'] = '0'
rdnet_in['SHIPPINGCARRIER'] = '0'
rdnet_in['STATUS'] = 'Error'
rdnet_in['STOPTYPE'] = 'stpStop'
rdnet_in['TOTALDISTANCE'] = '.000000'
rdnet_in['TOTALROUTEDISTANCE'] = '.000000'
rdnet_in['TRUCKANDTRAILERASSIGNED'] = 'No'
rdnet_in['UNITID'] = ''
rdnet_in['VEHICLEID'] = 'TT4X2TAIL'
rdnet_in['STOPSERVICETIME'] = '720'
rdnet_in['TOTALSERVICETIME'] = '720'
rdnet_in['TOTALTRAVELTIME'] = '0'
rdnet_in['LINEREFID'] = rdnet_in['INVENTTRANSID']

In [32]:
date_txt = input('Enter the dispatch date in the format yyyy-mm-dd: ')
date_dt = pd.to_datetime(date_txt)
#date_dt = pd.to_datetime('today')

In [33]:
rdnet_in['ROUTECOMPLETETIME'] = date_dt
rdnet_in['ROUTECOMPLETETIME'] = rdnet_in['ROUTECOMPLETETIME'].dt.normalize() + pd.Timedelta(days=0) + pd.Timedelta(hours=13) + pd.Timedelta(minutes=19)

rdnet_in['ROUTESTARTTIME'] = date_dt
rdnet_in['ROUTESTARTTIME'] = rdnet_in['ROUTESTARTTIME'].dt.normalize() + pd.Timedelta(days=0) + pd.Timedelta(hours=4) + pd.Timedelta(minutes=0)

rdnet_in['SCHEDULEDARRIVALDATETIME'] = date_dt
rdnet_in['SCHEDULEDARRIVALDATETIME'] = rdnet_in['SCHEDULEDARRIVALDATETIME'].dt.normalize() + pd.Timedelta(days=0) + pd.Timedelta(hours=12) + pd.Timedelta(minutes=59)

rdnet_in['SCHEDULEDDELIVERYDATETIME'] = date_dt
rdnet_in['SCHEDULEDDELIVERYDATETIME'] = rdnet_in['SCHEDULEDDELIVERYDATETIME'].dt.normalize() + pd.Timedelta(days=0)

rdnet_in['SCHEDULEDSHIPDATETIME'] = date_dt
rdnet_in['SCHEDULEDSHIPDATETIME'] = rdnet_in['SCHEDULEDSHIPDATETIME'].dt.normalize() + pd.Timedelta(days=0) + pd.Timedelta(hours=4) + pd.Timedelta(minutes=10)

rdnet_in['STOPARRIVALTIME'] = date_dt
rdnet_in['STOPARRIVALTIME'] = rdnet_in['STOPARRIVALTIME'].dt.normalize() + pd.Timedelta(days=0) + pd.Timedelta(hours=8) + pd.Timedelta(minutes=28)

### Get customer master in order to get the postal code

In [34]:
print("Loading the customer master to obtain the zipcode")
#Do the following once-off to create the parquet file
#customers=pd.read_csv("./data/customers/2023-08-10_CustomersV3.csv",low_memory=False)
#customers.to_parquet("./data/customers.parquet")
customers=pd.read_parquet("./data/customers.parquet")
customers_short = customers[['ADDRESSZIPCODE','CUSTOMERACCOUNT','ORGANIZATIONNAME']].copy()
#customers_short=pd.read_csv("./data/customers/customers_short.csv",low_memory=False)
customers_short['ADDRESSZIPCODE'] = customers_short['ADDRESSZIPCODE'].fillna(0)
customers_short['ADDRESSZIPCODE'] = customers_short['ADDRESSZIPCODE'].astype(int)
customers_short['ADDRESSZIPCODE'] = customers_short['ADDRESSZIPCODE'].astype(str)

Loading the customer master to obtain the zipcode


In [35]:
len(rdnet_in)

3983

In [36]:
rdnet_in_bk=rdnet_in.copy()

In [37]:
# To restore
#rdnet_in=rdnet_in_bk.copy()

In [38]:
rdnet_in = pd.merge(
    rdnet_in,
    customers_short,
    how="inner",
    on=None,
    left_on='STOPLOCATIONID',
    right_on='CUSTOMERACCOUNT',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [39]:
#len(rdnet_in)

In [40]:
rdnet_in.rename(columns={'ADDRESSZIPCODE':'STOPPOSTALCODE'}, inplace=True)
rdnet_in.drop(columns={'CUSTOMERACCOUNT', 'ORGANIZATIONNAME'}, inplace=True, axis=1)

In [41]:
#rdnet_in['STOPPOSTALCODE']

In [42]:
#rdnet_in.sort_values(['ORIGINLOCATIONID','ROADNETROUTE']).to_csv('data/roadnet/try1.csv', index=False)

### Split file into n files, each with a different ORIGINLOCATIONID
- Read csv file with ORIGINLOCATIONIDs
- Generate a dataframe with the unique list of warehouses
- Add a column (Group) to that dataframe where the warehouses are assigned into n groups
- Generate n output files, each with ORIGINLOCATIONID as from the csv file

In [43]:
input('Remember to update data/roadnet/sessionIDs.csv with the list of session IDs.  This will determine how the Roadnet inbound file will be split.  Press "Enter" to continue. ')
sessionids = pd.read_csv('data/roadnet/sessionIDs.csv')

In [44]:
print("Splitting inbound files per sessionID")

Splitting inbound files per sessionID


In [45]:
sessionids_list = sessionids.values.tolist()

In [46]:
unique_warehouses_np = rdnet_in['ORIGINLOCATIONID'].unique()
unique_warehouses_df = pd.DataFrame(unique_warehouses_np)
number_of_warehouses = len(rdnet_in['ORIGINLOCATIONID'].unique())
number_of_sessionIDs = len(sessionids)

In [47]:
number_of_warehouses

59

In [48]:
if number_of_warehouses < number_of_sessionIDs:
    raise SystemExit("There are more sessionIDs than warehouses - some of the output files will have no lines")

In [49]:
rdnet_in['WH_route_combination'] = rdnet_in['ORIGINLOCATIONID'].astype(str) + rdnet_in['ROADNETROUTE'].astype(str)

In [50]:
# New code to do a more equal distrobution of warehouses (itv of lines) over the sessions. 
# If there are 8 sessionIDs (i.e. virtual users), do it the following way:
# Step through warehouses from one with most lines to one with fewest lines
# Assign sequentially to the users 1 to 8
# Then, assign the remaining in reverse order from 8 to 1, continue allocating in descending quantity. Wrap back to 8 if needed.

#x = rdnet_in.groupby(['ORIGINLOCATIONID','ROADNETROUTE']).agg({'INVENTTRANSID': 'count', 'WH_route_combination': 'first'}).sort_values('INVENTTRANSID',ascending=False).reset_index().copy()
x = rdnet_in.groupby(['WH_route_combination']).agg({'INVENTTRANSID': 'count'}).sort_values('INVENTTRANSID',ascending=False).reset_index().copy()


In [51]:
x['sessionID'] = ''
user = 0
user_ascending = True
for i in range(len(x)):
    if user_ascending == True and user < number_of_sessionIDs:
        user = user + 1

    if user_ascending == False and user <= number_of_sessionIDs:
        if user > 1:
            user = user - 1
        else:
            user = number_of_sessionIDs

    if user_ascending == True and user == number_of_sessionIDs:
        user_ascending = False
        user = number_of_sessionIDs           

    x.at[i, 'sessionID'] = str(sessionids_list[user-1][0])

In [52]:
x.drop(columns={'INVENTTRANSID'}, inplace=True, axis=1)

### Merge back the warehouse-based split into the dataframe

In [53]:
len(rdnet_in)

3240

In [54]:
rdnet_in = pd.merge(
    rdnet_in,
    x,
    how="inner",
    on=None,
    left_on='WH_route_combination',
    right_on='WH_route_combination',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [55]:
len(rdnet_in)

3240

In [56]:
rdnet_in.drop(columns={'WH_route_combination'}, inplace=True, axis=1)
rdnet_in.rename(columns={'sessionID':'DYNAMICSRETRIEVALSESSIONID'}, inplace=True)

### Generate CSV files

In [57]:
print("Writing CSV files per sessionID")
# Clean up the folder where the inbound files are to be stored
path = "data/roadnet/inbound/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)  

Writing CSV files per sessionID


In [58]:
for sessionID in sessionids_list:
    mask = (rdnet_in['DYNAMICSRETRIEVALSESSIONID'] == sessionID[0])
    df_temp = rdnet_in[mask].copy()
    filename = 'data/roadnet/inbound/rdnet_inbound_'+sessionID[0]
    df_temp.to_csv(filename+'.csv',index=False, lineterminator='\r\n')    
    df_temp.to_xml(filename+'.xml',index=False)    

In [59]:
print('The Roadnet order lines were split by warehouse and route ID, into ' + str(number_of_sessionIDs) + ' files.  The lines were split as follows:')
print(rdnet_in.groupby(['DYNAMICSRETRIEVALSESSIONID']).agg({'INVENTTRANSID': 'count'}))


The Roadnet order lines were split by warehouse and route ID, into 5 files.  The lines were split as follows:
                            INVENTTRANSID
DYNAMICSRETRIEVALSESSIONID               
ZA1-000007781                         697
ZA1-000007782                         745
ZA1-000007783                         645
ZA1-000007784                         677
ZA1-000007785                         476


### End of script